In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
df = pd.read_csv("../input/plant-pathology-2020-fgvc7/train.csv")
df

In [ ]:
categories = {"healthy" : 0, "multiple_diseases" : 1, "rust" : 2, "scab" : 3}

def category(healthy, multiple_diseases, rust, scab):
    if healthy == 1:
        return "healthy"
    elif rust == 1:
        return "rust"
    elif scab == 1:
        return "scab"
    return "multiple_diseases"

df["label"] = df.apply(lambda x : category(x.healthy, x.multiple_diseases, x.rust, x.scab) , axis = 1)
df

In [ ]:
df["file_name"] = df.image_id + ".jpg"
df

In [ ]:
from tensorflow import keras

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.1,
    zoom_range=0.1,
    rotation_range = 0.1,
    horizontal_flip=True,
    vertical_flip = True,
    validation_split=0.1)

In [ ]:
train_data = datagen.flow_from_dataframe(df , x_col="file_name", y_col = "label", 
                                        directory = "../input/plant-pathology-2020-fgvc7/images", 
                                        target_size = (512, 512), 
                                        batch_size = 32, subset = "training")

test_data = datagen.flow_from_dataframe(df , x_col="file_name", y_col = "label", 
                                        directory = "../input/plant-pathology-2020-fgvc7/images", 
                                        target_size = (512, 512), 
                                        batch_size = 32, subset = "validation")

In [ ]:
base_model = keras.applications.ResNet50V2(
    include_top = False,
    weights="imagenet",
)

In [ ]:
base_model.trainable = False

In [ ]:
model = keras.Sequential([
    base_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(4, activation='softmax')
])

model.compile(
    optimizer="Adam",
    loss='categorical_crossentropy',
    metrics=["accuracy"]
)

history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=50,
    callbacks = [keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.001)]
)

In [ ]:
df = pd.read_csv("../input/plant-pathology-2020-fgvc7/test.csv")
df

In [ ]:
df.image_id = df.image_id + ".jpg"

In [ ]:
df["label"] = df.apply(lambda x : "0", axis = 1)

In [ ]:
df

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.1,
    zoom_range=0.1,
    rotation_range = 0.1,
    horizontal_flip=True,
    vertical_flip = True,
    validation_split=0)
test_data = datagen.flow_from_dataframe(df , x_col="image_id", y_col = "label",
                                        directory = "../input/plant-pathology-2020-fgvc7/images", 
                                        target_size = (512, 512), 
                                        batch_size = 32, subset = "training")

In [ ]:
pred = model.predict_classes(test_data)

In [ ]:
df["pred"] = pred
df

In [ ]:
df["healthy"] = df.apply(lambda x : 1 if x["pred"] == 0 else 0, axis = 1)
df["multiple_diseases"] = df.apply(lambda x : 1 if x["pred"] == 1 else 0, axis = 1)
df["rust"] = df.apply(lambda x : 1 if x["pred"] == 2 else 0, axis = 1)
df["scab"] = df.apply(lambda x : 1 if x["pred"] == 3 else 0, axis = 1)
df

In [ ]:
df = df.drop(columns = ["pred", "label"])
df

In [ ]:
df.to_csv("a.csv",index = False)